In [42]:
import pandas as pd
import geopandas as gpd
from scipy import spatial
from geopy.distance import vincenty
import requests
import geojson

In [3]:
df_school = pd.read_csv("../datagoid/all_school.csv")

/Users/ryanurzha/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
df_school['coord'] = df_school.apply(lambda x: (x['lintang'],x['bujur']),axis=1)

In [56]:
df_school.head()

,Unnamed: 0,Unnamed: 0.1,alamat_jalan,bentuk,bujur,id,kabkode,kabupaten_kota,kecamatan,kode_kab_kota,kode_kec,kode_prop,level,lintang,npsn,propinsi,sekolah,status,coord
0,0,0.0,Pulau Panggang Rt. 07/03,SD,106.6025,002A9C94-2BF5-E011-BFDE-6FB199584CC5,10100.0,Kab. Kepulauan Seribu,Kec. Kepulauan Seribu Utara,10100.0,10102.0,10000.0,sd,-5.7397,20104468,Prop. D.K.I. Jakarta,SD NEGERI PULAU PANGGANG 01 PG,N,"(-5.7397, 106.6025)"
1,1,1.0,Pulau Harapan Rt. 04/02,SD,106.5798,100C9C94-2BF5-E011-B842-FB97883C6736,10100.0,Kab. Kepulauan Seribu,Kec. Kepulauan Seribu Utara,10100.0,10102.0,10000.0,sd,-5.6536,20104464,Prop. D.K.I. Jakarta,SD NEGERI PULAU HARAPAN 01 PG,N,"(-5.6536, 106.5798)"
2,2,2.0,Pulau Panggang Rt. 07/03,SD,106.6024,20659C94-2BF5-E011-906F-335FFCD534E6,10100.0,Kab. Kepulauan Seribu,Kec. Kepulauan Seribu Utara,10100.0,10102.0,10000.0,sd,-5.7397,20104470,Prop. D.K.I. Jakarta,SD NEGERI PULAU PANGGANG 03 PG JAKARTA,N,"(-5.7397, 106.6024)"
3,3,3.0,Pulau Payung Rt. 06/04,SD,106.5549,80FD9B94-2BF5-E011-8604-71F029F60680,10100.0,Kab. Kepulauan Seribu,Kec. Kepulauan Seribu Selatan,10100.0,10101.0,10000.0,sd,-5.8223,20104476,Prop. D.K.I. Jakarta,SD NEGERI PULAU TIDUNG 04 PG,N,"(-5.8223, 106.5549)"
4,4,4.0,Jl. Pendidikan Rt. 07/02,SD,106.4926,50D19B94-2BF5-E011-8BCB-579EB8758510,10100.0,Kab. Kepulauan Seribu,Kec. Kepulauan Seribu Selatan,10100.0,10101.0,10000.0,sd,-5.7964,20104475,Prop. D.K.I. Jakarta,SD NEGERI PULAU TIDUNG 03 PG,N,"(-5.7964, 106.4926)"


In [57]:
df_jakarta = df_school[df_school.kode_prop==10000]

In [58]:
d_sd = df_jakarta[df_school.level=='sd'][['npsn','coord']].to_dict(orient='records')
d_smp = df_jakarta[df_school.level=='smp'][['npsn','coord']].to_dict(orient='records')
d_sma = df_jakarta[df_school.level=='sma'][['npsn','coord']].to_dict(orient='records')

/Users/ryanurzha/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/Users/ryanurzha/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/Users/ryanurzha/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [59]:
len(d_smp)

1096

In [60]:
randompoint = (-6.265795, 106.820634)

In [61]:
for i in d_smp[0:5]:
    print(i['coord'])

(-5.7985, 106.5003)
(-5.8578, 106.6196)
(-5.7772, 106.5053)
(-5.7471, 106.6138)
(-5.9788, 106.7098)


In [64]:
def findnearest(point,num,poi_list):
    #using vincenty great circle distance
    #overkill i kno
    for p in poi_list:
        p['dist'] = vincenty(p['coord'],point).meters
    return(sorted(poi_list, key=lambda x: x['dist'])[0:num])

In [66]:
top5_smp = findnearest(randompoint,5,d_smp)

In [67]:
top5_smp

[{'coord': (-6.2649, 106.8239),
  'dist': 374.71998113342534,
  'npsn': '20102508'},
 {'coord': (-6.272, 106.8216), 'dist': 694.4717739444661, 'npsn': '69892782'},
 {'coord': (-6.2565, 106.8209), 'dist': 1028.332036309646, 'npsn': '69756890'},
 {'coord': (-6.258036, 106.814097),
  'dist': 1122.2875411121026,
  'npsn': '69889002'},
 {'coord': (-6.2601, 106.8299),
  'dist': 1203.3405144733151,
  'npsn': '20108330'}]